# Parallel Factor Run (Modular)

Compute factors in parallel, then reuse results for analytics/correlations/rolling without recomputing factors.

In [1]:
from datetime import date
import logging
import sys
from pathlib import Path

try:
    root = Path(__file__).resolve().parents[1]
except NameError:
    root = Path.cwd().resolve().parents[0]
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

from quantlab_factor_library.run_factors import (
    compute_factors,
    run_analytics_only,
    compute_correlations_only,
    run_time_effects,
)
logging.basicConfig(level=logging.INFO)


## 1) Compute factors in parallel

In [2]:
# Adjust workers to your CPU/I/O; 4 is a safer default than 10
factors, ls_returns, ff, fwd_returns = compute_factors(parallel=True, max_workers=4)
print(f"Computed {len(factors)} factors; LS series: {len(ls_returns)}; FF available: {ff is not None}")


INFO:quantlab_factor_library.run_factors:Saved FF factors to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_timeseries.parquet
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/industry_co_momentum.py:39: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_factor_library/factors/residual_momentum.py:29: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()
/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/quantlab_f

Computed 58 factors; LS series: 58; FF available: True


## 2) Analytics

In [3]:
analytics = run_analytics_only(factors, fwd_returns, ff=ff, write_registry=True)
{k: {k2: res['summary'].get(k2) for k2 in ['ls_sharpe','ls_max_drawdown','ls_sharpe_last_yr','ls_max_drawdown_last_yr']} for k,res in analytics.items()}


INFO:quantlab_factor_library.analytics:Updated factor registry at /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_analytics_summary.parquet
INFO:quantlab_factor_library.analytics:Updated diagnostics reference registry at /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.parquet and /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.csv
INFO:quantlab_factor_library.analytics:Updated factor registry at /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_analytics_summary.parquet
INFO:quantlab_factor_library.analytics:Updated diagnostics reference registry at /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.parquet and /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_analytics_summary.csv
INFO:quantlab_factor_library.analytics:Updated factor registry at /Users/edl/Documents/dev/quantlab_v2/data/factors/

{'industry_co_momentum': {'ls_sharpe': np.float64(-0.009153022659849807),
  'ls_max_drawdown': np.float64(-0.5454036373775568),
  'ls_sharpe_last_yr': np.float64(-0.8228849554634611),
  'ls_max_drawdown_last_yr': np.float64(-0.2292913731794437)},
 'momentum_12m': {'ls_sharpe': np.float64(-0.20288048382866425),
  'ls_max_drawdown': np.float64(-0.9172728115855876),
  'ls_sharpe_last_yr': np.float64(-0.5747597127065099),
  'ls_max_drawdown_last_yr': np.float64(-0.24287074540656295)},
 'residual_momentum_12m': {'ls_sharpe': np.float64(0.3549690172940553),
  'ls_max_drawdown': np.float64(-0.7462236851281956),
  'ls_sharpe_last_yr': np.float64(0.3229348467299529),
  'ls_max_drawdown_last_yr': np.float64(-0.1481054976329238)},
 'volume_inclusive_icm': {'ls_sharpe': np.float64(-0.22785105058830177),
  'ls_max_drawdown': np.float64(-0.7099085741866342),
  'ls_sharpe_last_yr': np.float64(-0.6171014861814759),
  'ls_max_drawdown_last_yr': np.float64(-0.25140681913105634)},
 'composite_momentum': 

## 3) Correlations

In [4]:
corr_paths = compute_correlations_only(factors, ls_returns=ls_returns, ff=ff)
corr_paths

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_correlation.parquet and diagnostics/factor_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_correlation.parquet and diagnostics/factor_ff_correlation.(parquet,csv)


(PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_correlation.parquet'),
 PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_ff_correlation.parquet'))

## 4) Rolling time effects

In [5]:
rolling = run_time_effects(factors, fwd_returns, window=252, step=21)
rolling.head() if not rolling.empty else "No rolling stats"


INFO:quantlab_factor_library.run_factors:Saved rolling analytics to /Users/edl/Documents/dev/quantlab_v2/data/factors/factor_rolling_analytics.parquet and diagnostics copies to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/factor_rolling_analytics.csv


,factor,date,rolling_mean_ic,rolling_ic_ir
0,industry_co_momentum,2002-02-08,0.011983,0.097442
1,industry_co_momentum,2002-03-12,0.007152,0.058175
2,industry_co_momentum,2002-04-11,0.006727,0.056294
3,industry_co_momentum,2002-05-10,0.006777,0.057339
4,industry_co_momentum,2002-06-11,0.010797,0.091929


# Composite factors (build, compare, pick best)
Steps: (1) build/diagnose composites under all weightings, (2) compare IC IR, (3) pick best weighting per composite, (4) save diagnostics for best set.


In [6]:
from quantlab_factor_library.composite import (
    run_composite_pipeline,
    ic_ir_map_from_analytics,
    ls_vol_map_from_analytics,
    select_best_weights,
)
import pandas as pd
# blend_12m=True uses 50/50 full IC IR and 12m IC for weighting
ic_map = ic_ir_map_from_analytics(analytics, blend_12m=True)
ls_vol_map = ls_vol_map_from_analytics(analytics)


In [7]:
# 1) Build + diagnose composites for each weighting
composite_runs = {}
for method in ["equal", "inv_vol", "ic_ir"]:
    res = run_composite_pipeline(
        factors, fwd_returns, ff=ff, ic_map=ic_map, ls_vol_map=ls_vol_map, weight_method=method, min_dates=60
    )
    composite_runs[method] = res
    print(f"{method}: composites={len(res['composites'])}, saved={res['paths']}")


INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


equal: composites=9, saved={'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')}, 'theta_sh

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


inv_vol: composites=9, saved={'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')}, 'theta_

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


ic_ir: composites=9, saved={'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')}, 'theta_sh

In [8]:
# 2) Compare IC IR across weightings
def extract_ic_ir(run):
    rows = []
    for name, res in run['analytics'].items():
        rows.append({"composite": name, "ic_ir": res.get('summary', {}).get('ic_ir')})
    return pd.DataFrame(rows).set_index('composite')

ic_tables = []
for method, run in composite_runs.items():
    df = extract_ic_ir(run)
    df.columns = [method]
    ic_tables.append(df)
summary_ic = pd.concat(ic_tables, axis=1)
summary_ic


,equal,inv_vol,ic_ir
composite,,,
theta_momentum_riskadj,0.064007,0.056930,0.056669
theta_pure_value,0.001676,0.004288,0.095377
theta_pure_quality,0.133987,0.142522,0.146335
theta_shortterm_reversal,0.028350,0.037261,-0.021210
theta_growth_acceleration,0.058937,0.061795,0.075338
theta_info_forensic_drift,-0.027587,-0.016994,0.009919
theta_structural_liquidity,0.091225,0.090004,0.107093
theta_systematic_risk,0.028925,0.031064,0.041290
theta_size_smallcap,-0.009357,-0.008558,-0.015557


In [9]:
# 3) Pick best weighting per composite based on IC IR
best_choice = select_best_weights(composite_runs, criterion='ic_ir')
best_choice


{'theta_momentum_riskadj': 'equal',
 'theta_pure_value': 'ic_ir',
 'theta_pure_quality': 'ic_ir',
 'theta_shortterm_reversal': 'inv_vol',
 'theta_growth_acceleration': 'ic_ir',
 'theta_info_forensic_drift': 'ic_ir',
 'theta_structural_liquidity': 'ic_ir',
 'theta_systematic_risk': 'ic_ir',
 'theta_size_smallcap': 'inv_vol'}

In [10]:
# 4) Re-run best-weight composites and save diagnostics
best_runs = {}
for comp, method in best_choice.items():
    res = run_composite_pipeline(
        factors, fwd_returns, ff=ff, ic_map=ic_map, ls_vol_map=ls_vol_map, weight_method=method, min_dates=60
    )
    if comp in res['composites']:
        best_runs[comp] = res
        print(f"{comp} -> {method}, saved {res['paths']}")
best_summary = []
for comp, res in best_runs.items():
    a = res['analytics'].get(comp, {})
    best_summary.append({"composite": comp, **(a.get('summary') or {})})
best_summary_df = pd.DataFrame(best_summary)
best_summary_df


INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


theta_momentum_riskadj -> equal, saved {'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


theta_pure_value -> ic_ir, saved {'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')}, 'th

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


...

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


theta_systematic_risk -> ic_ir, saved {'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')}

INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet and diagnostics/composite_correlation.(parquet,csv)
INFO:quantlab_factor_library.analytics:Saved factor correlation matrix to /Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet and diagnostics/composite_ff_correlation.(parquet,csv)


theta_size_smallcap -> inv_vol, saved {'summary': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.parquet'), 'csv': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_analytics_summary.csv')}, 'correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_correlation.parquet'), 'ff_correlation': PosixPath('/Users/edl/Documents/dev/quantlab_v2/quantlab_factor_library/diagnostics/composite_ff_correlation.parquet'), 'factor_files': {'theta_momentum_riskadj': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_momentum_riskadj.parquet')}, 'theta_pure_value': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_value.parquet')}, 'theta_pure_quality': {'parquet': PosixPath('/Users/edl/Documents/dev/quantlab_v2/data/factors/factor_theta_pure_quality.parquet')}

,composite,mean_ic,ic_tstat,ic_ir,ic_hit_rate,recent_ic_mean_60d,mean_autocorr,avg_decile_spread,avg_decile_0,avg_decile_1,...,ff_p_mktrf,ff_beta_smb,ff_t_smb,ff_p_smb,ff_beta_hml,ff_t_hml,ff_p_hml,ff_beta_umd,ff_t_umd,ff_p_umd
0,theta_momentum_riskadj,0.010376,4.957096,0.064007,0.544015,0.046230,0.792305,0.013683,0.001156,0.000803,...,0.019183,-0.029910,-0.947865,3.432366e-01,-0.001377,-0.050049,0.960085,0.121006,5.761493,8.752769e-09
1,theta_pure_value,0.007985,6.398767,0.095377,0.534992,0.025367,0.996767,0.012135,0.001113,0.000617,...,0.000030,-0.006013,-0.266415,7.899321e-01,-0.028183,-1.506936,0.131897,0.070357,4.781426,1.796355e-06
2,theta_pure_quality,0.013030,9.817526,0.146335,0.566319,0.018664,0.995628,0.010427,0.000770,0.000619,...,0.005895,-0.030444,-1.396978,1.624895e-01,0.020800,1.151872,0.249435,0.043618,3.070018,2.153269e-03
3,theta_shortterm_reversal,0.004162,2.977356,0.037261,0.503681,-0.029210,0.848046,0.012072,0.000606,0.000565,...,0.828892,0.006418,0.289090,7.725216e-01,0.002068,0.112066,0.910775,-0.091174,-6.416512,1.494613e-10
4,theta_growth_acceleration,0.006054,5.054381,0.075338,0.534326,0.019707,0.994355,0.009735,0.000616,0.000595,...,0.002854,0.075006,5.223942,1.830975e-07,-0.015458,-1.299256,0.193923,0.010302,1.100620,2.711212e-01
5,theta_info_forensic_drift,0.003528,0.213665,0.009919,0.487069,0.010174,NaN,0.037123,0.003676,0.001710,...,0.180505,-0.136467,-0.866127,3.868831e-01,0.043998,0.434573,0.664081,0.078460,0.975017,3.300766e-01
6,theta_structural_liquidity,0.008502,7.622459,0.107093,0.548954,0.006263,0.879975,0.011002,0.000929,0.000689,...,0.745514,-0.034606,-1.596763,1.103811e-01,-0.012949,-0.705968,0.480240,0.044571,3.132097,1.745554e-03
7,theta_systematic_risk,0.007289,2.770098,0.041290,0.511220,0.022611,0.998911,0.013791,0.001365,0.000720,...,0.382947,-0.016915,-0.482632,6.293804e-01,0.029448,1.014024,0.310626,0.086567,3.788598,1.534862e-04
8,theta_size_smallcap,-0.000907,-0.609147,-0.008558,0.502961,-0.038382,0.999818,0.010225,0.000515,0.000529,...,0.279326,-0.005452,-0.269273,7.877304e-01,-0.024770,-1.445695,0.148325,-0.036526,-2.747695,6.022893e-03
